In [73]:
from deepchem.molnet import load_bace_classification, load_bbbp
import numpy as np

from simcse import SimCSE
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from args_parser import parse_args
import sys
import pandas as pd

datasets = {
        "bace": load_bace_classification,
        "bbbp": load_bbbp
        }

sys.argv = ['']
args = parse_args()

args.samples_per_class=100
args.n_augment = 2
args.batch_size = 8

input_dim = 512
output_dim = args.num_labels
set_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



12/23/2022 13:48:49 - INFO - deepchem.molnet.load_function.molnet_loader -   About to featurize bace_c dataset.
12/23/2022 13:48:49 - INFO - deepchem.data.data_loader -   Loading raw samples now.
12/23/2022 13:48:49 - INFO - deepchem.data.data_loader -   shard_size: 8192
12/23/2022 13:48:49 - INFO - deepchem.utils.data_utils -   About to start loading CSV from /var/folders/s4/4l6cbdrn7cq2m4vs14xz9qpm0000gn/T/bace.csv
12/23/2022 13:48:49 - INFO - deepchem.utils.data_utils -   Loading shard 1 of size 8192.
12/23/2022 13:48:49 - INFO - deepchem.data.data_loader -   About to featurize shard.
12/23/2022 13:48:49 - INFO - deepchem.feat.base_classes -   Featurizing datapoint 0
12/23/2022 13:48:55 - INFO - deepchem.feat.base_classes -   Featurizing datapoint 1000
12/23/2022 13:48:58 - INFO - deepchem.data.data_loader -   TIMING: featurizing shard 0 took 9.081 s
12/23/2022 13:48:59 - INFO - deepchem.data.datasets -   TIMING: dataset construction took 9.270 s
12/23/2022 13:48:59 - INFO - deepche

In [74]:

model = SimCSE("shahrukhx01/muv2x-simcse-smole-bert")




Some weights of the model checkpoint at shahrukhx01/muv2x-simcse-smole-bert were not used when initializing BertModel: ['mlp.dense.bias', 'mlp.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at shahrukhx01/muv2x-simcse-smole-bert and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 3/3 [00:09<00:00,  3.14s/it]


In [75]:
"""train_idx = [_idx for _idx in range(len(train_embeddings))]
augmented_embeds, augmented_labels, lamdas = [], [], []
if args.n_augment:
    for indices_batch in batch(train_idx, args.batch_size):
        print(len(indices_batch))
        for i in range(args.n_augment):
            np.random.seed()
            lam = np.random.beta(args.alpha, args.alpha)
            embedding2_idx = np.random.choice(train_embeddings_idx, replace=False)
            embedding2 = train_embeddings[embedding2_idx, :]
            label2 = train_labels[embedding2_idx]
            aug_embed, aug_label =  mixup_augment(embedding1=train_embedding, embedding2=embedding2, label1=train_label+1, label2=label2+1, lamda=lam)
            aug_label = aug_label-1
            augmented_embeds.append(aug_embed)
            augmented_labels.append(aug_label)

train_embeddings_augmented, train_labels_augmented = None, None
if len(augmented_embeds):
    augmented_embeds = torch.stack(augmented_embeds)
    train_embeddings_augmented = torch.cat([train_embeddings, augmented_embeds])
    train_labels_augmented = train_labels + augmented_labels
else:
    train_embeddings_augmented, train_labels_augmented = train_embeddings, train_labels
train_embeddings_augmented.shape"""

'train_idx = [_idx for _idx in range(len(train_embeddings))]\naugmented_embeds, augmented_labels, lamdas = [], [], []\nif args.n_augment:\n    for indices_batch in batch(train_idx, args.batch_size):\n        print(len(indices_batch))\n        for i in range(args.n_augment):\n            np.random.seed()\n            lam = np.random.beta(args.alpha, args.alpha)\n            embedding2_idx = np.random.choice(train_embeddings_idx, replace=False)\n            embedding2 = train_embeddings[embedding2_idx, :]\n            label2 = train_labels[embedding2_idx]\n            aug_embed, aug_label =  mixup_augment(embedding1=train_embedding, embedding2=embedding2, label1=train_label+1, label2=label2+1, lamda=lam)\n            aug_label = aug_label-1\n            augmented_embeds.append(aug_embed)\n            augmented_labels.append(aug_label)\n\ntrain_embeddings_augmented, train_labels_augmented = None, None\nif len(augmented_embeds):\n    augmented_embeds = torch.stack(augmented_embeds)\n    tr

In [76]:



class MolNet(nn.Module):
    """
    This class is created to specify the Neural Network on which vectorized datasets we have created previously
    is trained on, validated and later tested.
    It consist of one input layer, one output layer and multiple hidden layers.
    ...
    """
    def __init__(self, input_dim, output_dim, dropout=0.5):
        super(MolNet, self).__init__()
        # Layer definitions
        self.layers = nn.Sequential(
        nn.Linear(input_dim, 1024),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(128, output_dim)
        )

    def forward(self, x):
        # Forward pass
        return self.layers(x)

model_mlp = MolNet(input_dim=input_dim, output_dim=output_dim).to(set_device)
criterion = nn.CrossEntropyLoss().to(set_device)
optimizer = getattr(optim, "Adam")(model_mlp.parameters(), lr=args.lr)


from sklearn.metrics import roc_auc_score
from torch import sigmoid
from torch.nn.functional import softmax


def flat_auroc_score(preds, labels):
    """
    Function to calculate the roc_auc_score of our predictions vs labels
    """
    pred_flat = softmax(preds, dim=1)[:, 1]
    # labels_flat = np.argmax(labels, axis=1)
    return roc_auc_score(labels, pred_flat.detach().cpu().numpy())

In [77]:
set_device.type

'cpu'

In [78]:
def mixup_augment(embedding1, embedding2, label1, label2, lamda):
    embedding_output = lam * embedding1 + (1.0 - lam) * embedding2
    label_output = lam * label1 + (1.0 - lam) * label2
    return (embedding_output, label_output)

def get_perm(x):
        """get random permutation"""
        batch_size = x.size()[0]
        if args.cuda and torch.cuda.is_available():
            index = torch.randperm(batch_size).cuda()
        else:
            index = torch.randperm(batch_size)
        return index
def mixup_criterion_cross_entropy(pred, y_a, y_b, lam):
        return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [79]:
best_model = None
best_accuracy = 0.0
train_loss_history, recall_train_history = [], []
validation_loss_history, recall_validation_history = list(), list()
for epoch in range(0, args.epoch):
        model_mlp.train()
        train_loss_scores = []
        training_acc_scores = []
        y_pred, y_true= list(), list()
        predictions = []
        for x, y in train_dataloader:
            ## perform forward pass  
            x = x.type(torch.FloatTensor).to(set_device)
            y = y.type(torch.LongTensor).to(set_device)
            for i in range(args.n_augment):
                lam = np.random.beta(args.alpha, args.alpha)
                indices_permuted = get_perm(x)
                x2 = x[indices_permuted, :]
                y2 = y[indices_permuted]
                mixup_x, mixup_y = mixup_augment(embedding1=x, embedding2=x2, label1=y, label2=y2, lamda=lam)
            
                pred = model_mlp(mixup_x) 

                preds = torch.max(pred, 1)[1]

                ## accumulate predictions per batch for the epoch
                """y_pred += list([x.item() for x in preds.detach().cpu().numpy()])
                targets = torch.LongTensor([x.item() for x in list(targets)])
                y_true +=  list([x.item() for x in targets.detach().cpu().numpy()])"""

                ## compute loss and perform backward pass
                loss = mixup_criterion_cross_entropy(pred, y, y2, lam=lam) ## compute loss 
                optimizer.zero_grad()
                loss.backward() 
                optimizer.step()

                predictions.append(pred)

                ## accumulate train loss
                train_loss_scores.append(loss.item())

        ## accumulate loss, recall, f1, precision per epoch
        train_loss_history.append((sum(train_loss_scores)/len(train_loss_scores)))
        #recall = flat_auroc_score(predictions, y_true)
        #recall_train_history.append(recall)
        print(f'Training =>  Epoch : {epoch+1} | Loss : {train_loss_history[-1]}') 
          #| AUROC score: {recall_train_history[-1]}')
        
        model_mlp.eval()
        predictions = None
        with torch.no_grad():
            validation_loss_scores = list()
            y_true_val, y_pred_val= list(), list()

            ## perform validation pass
            for batch, targets in val_dataloader:
                ## perform forward pass  
                batch = batch.type(torch.FloatTensor).to(set_device)
                pred = model_mlp(batch) 
                predictions = pred
                preds = torch.max(pred, 1)[1]
                
                ## accumulate predictions per batch for the epoch
                y_pred_val += list([x.item() for x in preds.detach().cpu().numpy()])
                targets = torch.LongTensor([x.item() for x in list(targets)])
                y_true_val +=  list([x.item() for x in targets.detach().cpu().numpy()])
                
                ## computing validate loss
                loss = criterion(pred.to(set_device), targets.to(set_device)) ## compute loss 

                ## accumulate validate loss
                validation_loss_scores.append(loss.item())
                
            
            ## accumulate loss, recall, f1, precision per epoch
            validation_loss_history.append((sum(validation_loss_scores)/len(validation_loss_scores)))
            recall = flat_auroc_score(predictions, y_true_val)
            recall_validation_history.append(recall)

            print(f'Validation =>  Epoch : {epoch+1} | Loss : {validation_loss_history[-1]} | AUROC score: {recall_validation_history[-1]} ')
            
            if recall_validation_history[-1]>best_accuracy:
                best_accuracy = recall_validation_history[-1]
                print('Selecting the model...')
                best_model = model_mlp


Training =>  Epoch : 1 | Loss : 0.698230789899826
Validation =>  Epoch : 1 | Loss : 0.6938460469245911 | AUROC score: 0.619047619047619 
Selecting the model...


<ipython-input-79-6be130c99b77>:65: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  targets = torch.LongTensor([x.item() for x in list(targets)])


Training =>  Epoch : 2 | Loss : 0.6901604342460632
Validation =>  Epoch : 2 | Loss : 0.6785762310028076 | AUROC score: 0.6293532338308457 
Selecting the model...


<ipython-input-79-6be130c99b77>:65: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  targets = torch.LongTensor([x.item() for x in list(targets)])


Training =>  Epoch : 3 | Loss : 0.6690832436084747
Validation =>  Epoch : 3 | Loss : 0.7490429282188416 | AUROC score: 0.6663113006396588 
Selecting the model...


...

<ipython-input-79-6be130c99b77>:65: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  targets = torch.LongTensor([x.item() for x in list(targets)])


Training =>  Epoch : 19 | Loss : 0.4824518275260925
Validation =>  Epoch : 19 | Loss : 0.7567782402038574 | AUROC score: 0.7121535181236673 


<ipython-input-79-6be130c99b77>:65: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  targets = torch.LongTensor([x.item() for x in list(targets)])


Training =>  Epoch : 20 | Loss : 0.4674354638159275
Validation =>  Epoch : 20 | Loss : 0.6789330244064331 | AUROC score: 0.7032693674484719 


<ipython-input-79-6be130c99b77>:65: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  targets = torch.LongTensor([x.item() for x in list(targets)])


In [80]:
best_model.eval()
predictions = None
with torch.no_grad():
    validation_loss_scores = list()
    y_true_val, y_pred_val= list(), list()

    ## perform validation pass
    for batch, targets in test_dataloader:
        ## perform forward pass  
        batch = batch.type(torch.FloatTensor).to(set_device)
        pred = best_model(batch) 
        predictions = pred
        preds = torch.max(pred, 1)[1]

        ## accumulate predictions per batch for the epoch
        y_pred_val += list([x.item() for x in preds.detach().cpu().numpy()])
        targets = torch.LongTensor([x.item() for x in list(targets)])
        y_true_val +=  list([x.item() for x in targets.detach().cpu().numpy()])

        ## computing validate loss
        loss = criterion(pred.to(set_device), targets.to(set_device)) ## compute loss 

        ## accumulate validate loss
        validation_loss_scores.append(loss.item())


    ## accumulate loss, recall, f1, precision per epoch
    validation_loss_history.append((sum(validation_loss_scores)/len(validation_loss_scores)))
    recall = flat_auroc_score(predictions, y_true_val)
    recall_validation_history.append(recall)

    print(f'Test => AUROC score: {recall_validation_history[-1]} ')

Test => AUROC score: 0.6110507246376812 


<ipython-input-80-f9938ec2c9f4>:17: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  targets = torch.LongTensor([x.item() for x in list(targets)])
